In [5]:
import sqlite3
import pandas as pd

db = sqlite3.connect('AMS.db')
query = 'select * from Order_Table'
df = pd.read_sql(query, con = db)
df['Date']=pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
dfMonth = df[(df['Year']==2019) & (df['Month']==12)]
dfOrderPL = dfMonth.groupby(['Date','OrderID'])[['SettleAmt']].sum()
dfOrderPL.reset_index()

,Date,OrderID,SettleAmt
0,2019-12-02,1,491.6
1,2019-12-02,2,101.6
2,2019-12-03,1,1691.6
3,2019-12-05,1,-564.4
4,2019-12-05,2,345.6
5,2019-12-05,3,95.6
6,2019-12-05,4,918.4
7,2019-12-09,1,975.6
8,2019-12-09,2,-774.4
9,2019-12-09,3,-554.4


In [2]:
dfMonth.groupby(['Date'])[['SettleAmt']].sum()

,SettleAmt
Date,
2019-12-02,593.2
2019-12-03,1691.6
2019-12-05,795.2
2019-12-09,-353.2
2019-12-10,171.2


In [3]:
df.groupby(['Year','Month'])[['SettleAmt']].sum()

SettleAmt
Year Month           
2019 11        2037.6
     12        2898.0

In [10]:
import pandas as pd
import portfolio
import sqlite3

db = sqlite3.connect('AMS.db')

dt = '2019-10-13'
account = ('CMS','CITIC')
df_trans = portfolio.get_trans(dt, account)

query = "SELECT SymbolCode, AssetClass FROM Symbol_Table"
df_symbol = pd.read_sql(query, con=db)

df = pd.merge(df_trans, df_symbol, on='SymbolCode', how='left')

df_stock = df[~df['AssetClass'].isin(['CASH'])]
#df_stock[['SymbolCode','SymbolName']]
df_stock = df_stock[['AccountID','SymbolCode','SymbolName','CurSettle','Qty']]
df_stock_group = df_stock.groupby(['AccountID','SymbolCode', 'SymbolName', 'CurSettle']).sum()
df_stock_group

Qty
AccountID SymbolCode SymbolName CurSettle         
CITIC     000726.SZ  鲁泰A        CNY            0.0
          002294.SZ  信立泰        CNY         1000.0
          510500.SS  500ETF     CNY         5000.0
          512880.SS  证券ETF      CNY        10000.0
          600177.SS  雅戈尔        CNY         3800.0
          601088.SS  中国神华       CNY         4888.0
CMS       200625.SZ  长安B        HKD        11900.0
          512880.SS  证券ETF      CNY        48500.0
          600030.SS  中信证券       CNY         1350.0
          601166.SS  兴业银行       CNY         4732.0